<a href="https://colab.research.google.com/github/phisan-chula/Construction_Survey/blob/main/GeodComp_BasicCommand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**การใช้สคริปซ์ (script) ในการคำนวนงานรังวัดสำรวจ**<br>
รศ.ดร.ไพศาล สันติธรรมนนท์<br>
คณะวิศวกรรมศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย<br>
email : phisan.chula@gmail.com<br>
<br>
การใช้คำสั่ง GeographicLib ผ่าน Colab VM<br>
ลักษณะเดียวกันผู้ใช้สามารถติดตั้ง GeographicLib บนวินโดว์ แม็คบุค <br>
https://sourceforge.net/projects/geographiclib/files/distrib/



#**1) การแปลงปริมาณมุม Decimail Degree (DD) ไปเป็น Deg-Min-Sec (DMS) และแปลงกลับ**



In [6]:
%%capture
! pip install PyGeodesy
! apt-get install geographiclib-tools

In [7]:
from pygeodesy import dms

print( dms.toDMS( 120.4567890123 , prec=9 ) )

print(  dms.parseDMS( '   120°27′24.44044428″  ' ) )
print( dms.parseDMS( '   120:27:24.44044428  ', sep=':' ) )

120°27′24.44044428″
120.4567890123
120.4567890123


#**2)  การแปลงระบบพิกัด ECEF (XYZ) และพิกัดยีออเดติกซ์ ( ϕ λ h )** 

In [8]:
! CartConvert --input-string "14.0000 100.0000 10.000"

-1074865.112613 6095862.971702 1532984.248725


In [9]:
! CartConvert -r --input-string "-1074865.112613 6095862.971702 1532984.248725"

14.00000000000 100.00000000000 10.000000


#**3) การแปลงผลต่างค่าพิกัด ECEF(XYZ)1 ไปยัง ECEF(XYZ)2 ไปสู่ระบบพิกัดท้องถิ่น East-North-Up (ENU)**

In [10]:
! CartConvert -l 14.00000   100.00000 10.000     --input-string "14.0001   100.0002   10.300"
! CartConvert -l 14:00:00    100:00:00   10.000   --input-string "14:00:00.1    100:00:00.2    10.300 "

21.606824 11.063956 0.299954
6.001897 3.073319 0.299996


#**4) การแปลงค่าพิกัดในระบบยูทีเอ็ม <br>**
ออฟชั่น -p 9   แสดงทศนิยม 9 ตำแหน่ง<br>
ออฟชั่น -c   คำนวณ meridian convergence และ scale factor<br>

In [11]:
! GeoConvert  -p 9  --input-string "47N 500000  1500000"
! GeoConvert  -: -p 6   --input-string "47N 500000  1500000"


13.56845127846726 99.00000000000000
13:34:06.4246025N 099:00:00.0000000E


In [12]:
! GeoConvert -u -p 3 --input-string "13.56845127846726 99.00000000000000"
! GeoConvert -u  -p 3 --input-string "13:34:06.4246025N 099:00:00.0000000E"

47n 500000.000 1500000.000
47n 500000.000 1500000.000


In [13]:
! GeoConvert  -c   --input-string "47N 500000  1500000"
! GeoConvert  -c   --input-string "13:34:06.4246025N 099:00:00.0000000E"

0.00000 0.9996000
0.00000 0.9996000


#**4) การแปลงค่าพิกัด Military Grud Referece System**<br>
ระบบพิกัดมีความยืดหยุ่นปรับความละเอียดเชิงพื้นที่ได้<br>
กำหนดใช้ในการระบุพืนที่การปกครอง ของ กรมการปกครอง<br>
อ้างอิง  https://en.wikipedia.org/wiki/Military_Grid_Reference_System<br>

In [14]:
! GeoConvert -m  -p -3  --input-string "47N 500000  1500000"
! GeoConvert -m  -p 0   --input-string "47N 500000  1500000"
! GeoConvert -m  -p +1   --input-string "47N 500000  1500000"

47PNR0000
47PNR0000000000
47PNR000000000000


# **5) การคำนวนเส้น ยีออเดซิก (geodetic line)**<br>
*ปัญหาคำนวนทางตรง (direct problem)* <br>
มีค่าพิกัด phi1, lam1 , faz1, s12<br>
*ปัญหาคำนวนย้อน  (indirect/ inverse problem)*<br>
มึค่าพิกัด phi1,lam1, phi2,lam2<br>
ต้องการหา  faz1, faz2, s12<b4>


In [15]:
! GeodSolve -i --input-string "14.00000N 100.00000E 15.00000000N 101.0000000E"
! GeodSolve -i --input-string "14:00:00.00000 100:00:00.0000 15:00:00.00000 101:00:00.000000"

44.12901926 44.37941483 154472.518
44.12901926 44.37941483 154472.518


In [16]:
! GeodSolve -i  -d --input-string "14.00000N 100.00000E 15.00000000N 101.0000000E"
! GeodSolve -i  -d  --input-string "14:00:00.00000 100:00:00.0000 15:00:00.00000 101:00:00.000000"

044d07'44.4693" 044d22'45.8934" 154472.518
044d07'44.4693" 044d22'45.8934" 154472.518


#**6)  การแปลงค่าระดับจาก Orthometric Height ไปมาสู่ Ellipsoidal Height**<br>
แบบจำลอง TGM2017 สามารถติดตั้งได้ บนเครื่อง PC ส่วนตน</br>
hae = height above ellipsoid</br>
msl = mean sea level (orthometric height)

In [17]:
%%capture
from google.colab import drive
drive.mount('/content/drive')
! apt-get install geographiclib-tools
! mkdir -p /usr/share/GeographicLib/geoids
! cp /content/drive/MyDrive/Geodesy_Cache/tgm2017-1.pgm   /usr/share/GeographicLib/geoids/tgm2017-1.pgm

In [18]:
! GeoidEval -n tgm2017-1 --input-string "14 100"
! GeoidEval -n tgm2017-1 --msltohae  --input-string "14 100  1.5"
! GeoidEval -n tgm2017-1 --haetomsl  --input-string "14 100  -30.7867"

-32.2897
14 100  -30.7897
14 100  1.5030


In [19]:
! GeoidEval -n tgm2017-1 --input-string "47N 500000 1500000"

-33.2920


#**7) รัศมีโลกบนพื้นผิวทรงรีชนิดต่างๆ**<br>

In [26]:
import pygeodesy as pgd

print( pgd.Ellipsoids.GRS80.toStr() )
print( pgd.Ellipsoids.WGS84.toStr() )

print( pgd.Ellipsoids.WGS84.rocPrimeVertical(lat=45) )
print( pgd.Ellipsoids.WGS84.rocMeridional(lat=45)  )
print( pgd.Ellipsoids.WGS84.rocGauss(lat=45)  )
print( pgd.Ellipsoids.WGS84.rocBearing( 45, 10 )  )   # at latitude , azimut 0...360

name='GRS80', a=6378137, b=6356752.31414035, f_=298.2572221, f=0.00335281, f2=0.00336409, n=0.00167922, e=0.08181919, e2=0.00669438, e22=0.0067395, e32=0.00335843, A=6367449.14577104, L=10001965.72923046, R1=6371008.77138012, R2=6371007.18088351, R3=6371000.78997414, Rbiaxial=6367453.634464, Rtriaxial=6372797.55593326
name='WGS84', a=6378137, b=6356752.31424518, f_=298.25722356, f=0.00335281, f2=0.00336409, n=0.00167922, e=0.08181919, e2=0.00669438, e22=0.0067395, e32=0.00335843, A=6367449.14582341, L=10001965.72931272, R1=6371008.77141506, R2=6371007.18091847, R3=6371000.79000916, Rbiaxial=6367453.63451633, Rtriaxial=6372797.5559594
6388838.290121148
6367381.815619549
6378101.030201018
6368026.699920825
